In [115]:
import sys,os,argparse
import numpy as np
from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import ParameterGrid
import sklearn
from sklearn.naive_bayes import BernoulliNB,MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  VotingClassifier, BaggingClassifier


## Amazon elec dataset of reviews.
### 1- negative review 
### 2- positive review

In [117]:
# Load the dataset:
train_reviews = []
train_labels =[]
test_reviews = []
test_labels =[]
lemm = WordNetLemmatizer()

stops = stopwords.words('english')
def load_dataset(review_path, label_path, all_reviews, all_labels):

        
    # Loading spam-emails:
    with open(review_path, 'r') as f:
        for line in f.readlines():  
            #print(line)
            #line = line.decode('utf8')
            
            # Preprocess
            # Convert all line in lower case. Tokenize the review
            line= line.replace(":-)", "%smile")
            line= line.replace(":-(", "%sad")
            line= word_tokenize(line.lower())
            clean =""
            #line = [word for word in line if word not in stops]
            #line=[lemm.lemmatize(word) for word in line]
    
            #remove words of lenght 1
            line= " ".join([word for word in line if  len(word)>1]) # and not word.isdigit()])
            #print(line)
            all_reviews.append(line)
        f.close()
        
    with open(label_path, 'r') as f:
        for line in f.readlines():  
            all_labels.append(line)
        f.close()

    print ("Total number of critics: ", len(all_reviews), len(all_labels))
path = "./elec/elec-25k-train.txt.tok"
label_path = "./elec/elec-25k-train.cat"


load_dataset(path, label_path,train_reviews , train_labels)
for i in range(3):
    print(train_labels[i], train_reviews[i])
path = "./elec/elec-test.txt.tok"
label_path = "./elec/elec-test.cat"

load_dataset(path, label_path,test_reviews , test_labels)


Total number of critics:  25000 25000
2
 why this rating some years ago when bought my hv20 did lot of research on tapes and the consensus overwhelmingly pointed to the panasonic amq family so far with over 50 tapes used am happy camper indeed no dropouts and great quality am true believer now and hence the 5-star `` love it `` rating tape types irrespective of the tape you decide to use learned that it was more important to be loyal to the type of lubricant used referred to by my friends as wet or dry tapes the panasonic belongs to the dry type family whereas sony tapes use wet lubricant for instance panasonic claims that this tape uses `` .. dry type lubricant that causes less video head wear 4m after 500h it also prevents head clogs and extends head life `` while some on the net argue that this distinction is an urban myth decided to trust my professional friends on this issue this difference is not material until you switch between the two types which can lead to clogged video head

In [94]:
print(len(train_reviews), len(test_labels))

25000 25000


In [100]:
train_reviews = []
train_labels =[]

path = "./elec/elec-05k-train.txt.tok"
label_path = "./elec/elec-05k-train.cat"
load_dataset(path, label_path,train_reviews , train_labels)

#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer( min_df=2, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)
X = vectorizer.fit_transform(train_reviews)
X_test= vectorizer.transform(test_reviews)

Total number of critics:  5000 5000


In [101]:
from sklearn.model_selection import cross_val_score


#combination of l1 and squared_hinge / hinge is not supported
svm_param= ParameterGrid([{'random_state':[0],'loss':['hinge','squared_hinge'],'C':[0.008,0.01,0.02,.1, .5 , 1.0], 'penalty':['l2']}])
#svc_param = ParameterGrid({'random_state':[0],'kernel':['rbf','poly'],'C':[.5, 1.0,2.0]})

best_param_model=[]
#get the best params for each model on validation set
def get_best_param(class_and_param, train_vectors, train_labels):  #, val_vectors, val_labels):
    for (classifier, params) in class_and_param:
        print("For classifier: ",classifier,":")
        best_mean_acc = 0
        best_params=None
        for param in params:
            #act_score = compute_f1(classifier(**param), train_vectors, train_labels,val_vectors, val_labels)
            clf = classifier(**param)
            scores = cross_val_score(clf, train_vectors, train_labels, cv=4)
            print("The scores on cross validation are:", scores)                                              

            print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
            mean_acc = scores.mean()
            if(mean_acc >best_mean_acc):
                best_mean_acc = mean_acc
                best_params= param
            print("With the following params :", param)
            print("Mean Accuracy achieved : ", mean_acc)
        best_param_model.append(best_params)     
        print("\nFinal best params: ", best_params, "\nBest mean accuracy score on cross validation: ", best_mean_acc, "\n")
        
class_and_param= [(LinearSVC, svm_param)]
get_best_param(class_and_param,X, train_labels) 


For classifier:  <class 'sklearn.svm.classes.LinearSVC'> :
The scores on cross validation are: [0.8096 0.8008 0.776  0.784 ]
Accuracy: 0.79 (+/- 0.03)
With the following params : {'C': 0.008, 'loss': 'hinge', 'penalty': 'l2', 'random_state': 0}
Mean Accuracy achieved :  0.7926
The scores on cross validation are: [0.8424 0.8208 0.8184 0.828 ]
Accuracy: 0.83 (+/- 0.02)
With the following params : {'C': 0.008, 'loss': 'squared_hinge', 'penalty': 'l2', 'random_state': 0}
Mean Accuracy achieved :  0.8274
The scores on cross validation are: [0.8096 0.8008 0.776  0.784 ]
Accuracy: 0.79 (+/- 0.03)
With the following params : {'C': 0.01, 'loss': 'hinge', 'penalty': 'l2', 'random_state': 0}
Mean Accuracy achieved :  0.7926
The scores on cross validation are: [0.8464 0.8224 0.82   0.8288]
Accuracy: 0.83 (+/- 0.02)
With the following params : {'C': 0.01, 'loss': 'squared_hinge', 'penalty': 'l2', 'random_state': 0}
Mean Accuracy achieved :  0.8293999999999999
The scores on cross validation are: [0.

In [105]:
print ("Unigram\n")
vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer( min_df=2, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)
X = vectorizer.fit_transform(train_reviews)
    
#print(vectorizer.get_feature_names())
#print(X.toarray())  
# Train SVM:
classifier =LinearSVC(C=0.5, loss='hinge' ,random_state=0)
classifier.fit(X, train_labels)

X_test= vectorizer.transform(test_reviews)
pred = classifier.predict(X_test)
print ("SVM\tAccuracy of test set: (train on 25K) ", sklearn.metrics.accuracy_score(test_labels, pred))  
    

Unigram

SVM	Accuracy of test set: (train on 25K)  0.88644


In [116]:
print ("Bigram\n")

#vectorizer = CountVectorizer(ngram_range=(1, 2))
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=2, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)
X = vectorizer.fit_transform(train_reviews)
    
#print(vectorizer.get_feature_names())
#print(X.toarray())  
# Train SVM:

X_test= vectorizer.transform(test_reviews)
#pred = classifier.predict(X_test)
#print ("SVM\tAccuracy of validation set: ", sklearn.metrics.accuracy_score(test_labels, pred))  
    
best_param_model=[]
#get the best params for each model on validation set
def get_best_param(class_and_param, train_vectors, train_labels):  #, val_vectors, val_labels):
    for (classifier, params) in class_and_param:
        print("For classifier: ",classifier,":")
        best_mean_acc = 0
        best_params=None
        for param in params:
            #act_score = compute_f1(classifier(**param), train_vectors, train_labels,val_vectors, val_labels)
            clf = classifier(**param)
            scores = cross_val_score(clf, train_vectors, train_labels, cv=3)
            print("The scores on cross validation are:", scores)                                              

            print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
            mean_acc = scores.mean()
            if(mean_acc >best_mean_acc):
                best_mean_acc = mean_acc
                best_params= param
            print("With the following params :", param)
            print("Mean Accuracy achieved : ", mean_acc)
        best_param_model.append(best_params)     
        print("\nFinal best params: ", best_params, "\nBest mean accuracy score on cross validation: ", best_mean_acc, "\n")
        
nb_params = ParameterGrid({'alpha':[1e-4, 1e-3,0.01,0.1,0.3,0.6,1,2,3,5]})
classifiers = [(MultinomialNB, nb_params)] #, (LinearSVC, svm_param)]
get_best_param(classifiers,X, train_labels) 


Bigram

For classifier:  <class 'sklearn.naive_bayes.MultinomialNB'> :
The scores on cross validation are: [0.825054   0.82397408 0.82273164]
Accuracy: 0.82 (+/- 0.00)
With the following params : {'alpha': 0.0001}
Mean Accuracy achieved :  0.8239199049385699
The scores on cross validation are: [0.84293257 0.84209263 0.84013442]
Accuracy: 0.84 (+/- 0.00)
With the following params : {'alpha': 0.001}
Mean Accuracy achieved :  0.8417198731638674
The scores on cross validation are: [0.86681066 0.86537077 0.86053769]
Accuracy: 0.86 (+/- 0.01)
With the following params : {'alpha': 0.01}
Mean Accuracy achieved :  0.8642397038385753
The scores on cross validation are: [0.88132949 0.88312935 0.88118099]
Accuracy: 0.88 (+/- 0.00)
With the following params : {'alpha': 0.1}
Mean Accuracy achieved :  0.8818799440835899
The scores on cross validation are: [0.88396928 0.8887689  0.88514162]
Accuracy: 0.89 (+/- 0.00)
With the following params : {'alpha': 0.3}
Mean Accuracy achieved :  0.88595993453505


In [120]:
print ("Bigram with following parameters:\n")
classifier = LinearSVC(C=1, loss='squared_hinge', penalty='l2', random_state=0)
classifier = MultinomialNB(alpha=0.4)
print(classifier)

#Vectorize
vectorizer2 = CountVectorizer(ngram_range=(1, 2))
#vectorizer2 = TfidfVectorizer(ngram_range=(1, 2), min_df=2, strip_accents='unicode', use_idf=1,
 #              smooth_idf=1, sublinear_tf=1)
X = vectorizer2.fit_transform(train_reviews)
    
# Train SVM:
classifier.fit(X, train_labels)

X_test= vectorizer2.transform(test_reviews)
pred = classifier.predict(X_test)
print ("SVM\tAccuracy of validation set: ", sklearn.metrics.accuracy_score(test_labels, pred))  
    

Bigram with following parameters:

MultinomialNB(alpha=0.4, class_prior=None, fit_prior=True)
SVM	Accuracy of validation set:  0.87216


In [89]:
print ("Trigram\n")
train_reviews = []
train_labels =[]

path = "./elec/elec-05k-train.txt.tok"
label_path = "./elec/elec-05k-train.cat"
load_dataset(path, label_path,train_reviews , train_labels)

vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=2, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)
X = vectorizer.fit_transform(train_reviews)
    
#print(vectorizer.get_feature_names())
#print(X.toarray())  
# Train SVM:

X_test= vectorizer.transform(test_reviews)
#pred = classifier.predict(X_test)
#print ("SVM\tAccuracy of validation set: ", sklearn.metrics.accuracy_score(test_labels, pred))  
    
best_param_model=[]
nb_params = ParameterGrid({'alpha':[1e-4, 1e-3,0.01,0.1,0.3,0.6,1,2,3]})
classifiers = [(BernoulliNB, nb_params), (LinearSVC, svm_param)]
get_best_param(classifiers,X, train_labels) 

Trigram

Total number of critics:  5000 5000
For classifier:  <class 'sklearn.naive_bayes.BernoulliNB'> :
The scores on cross validation are: [0.81594724 0.82472989 0.83733493]
Accuracy: 0.83 (+/- 0.02)
With the following params : {'alpha': 0.0001}
Mean Accuracy achieved :  0.8260040227122024
The scores on cross validation are: [0.82434053 0.83133253 0.83973589]
Accuracy: 0.83 (+/- 0.01)
With the following params : {'alpha': 0.001}
Mean Accuracy achieved :  0.831802984982962
The scores on cross validation are: [0.83513189 0.83913565 0.84273709]
Accuracy: 0.84 (+/- 0.01)
With the following params : {'alpha': 0.01}
Mean Accuracy achieved :  0.8390015478613507
The scores on cross validation are: [0.85191847 0.82653061 0.82653061]
Accuracy: 0.83 (+/- 0.02)
With the following params : {'alpha': 0.1}
Mean Accuracy achieved :  0.8349932299058712
The scores on cross validation are: [0.85791367 0.82292917 0.81452581]
Accuracy: 0.83 (+/- 0.04)
With the following params : {'alpha': 0.3}
Mean Accu

In [122]:
print ("Trigram with following parameters:\n")
classifier = LinearSVC(C=1, loss='hinge', penalty='l2', random_state=0)
#classifier = BernoulliNB(alpha=0.1)
classifier = MultinomialNB(alpha=0.4)

print(classifier)
#Vectorize
vectorizer2 = CountVectorizer(ngram_range=(1, 3))
#vectorizer2 = TfidfVectorizer(ngram_range=(1, 3), min_df=2, strip_accents='unicode', use_idf=1,
 #              smooth_idf=1, sublinear_tf=1)
X = vectorizer2.fit_transform(train_reviews)

bagclf=BaggingClassifier(classifier,max_samples=0.7)
bagclf.fit(X, train_labels)

# Train SVM:
classifier.fit(X, train_labels)

X_test= vectorizer2.transform(test_reviews)
pred = classifier.predict(X_test)
print ("SVM\tAccuracy of validation set: ", sklearn.metrics.accuracy_score(test_labels, pred))  
    

Trigram with following parameters:

MultinomialNB(alpha=0.4, class_prior=None, fit_prior=True)
SVM	Accuracy of validation set:  0.88364


In [35]:
#classifier.decision_function(test_reviews)
test_reviews
import pandas as pd
#dat1 = pd.DataFrame({'data': X_test})
#dat2 = pd.DataFrame({'labels': test_labels})

#dat=dat1.join(dat2)
#print(dat.values)
classifier.decision_function(X_test)[:20]

array([ 5.26964051, -0.57380242,  1.14079745,  0.93619764,  1.05929235,
        0.37334154,  2.41763543,  1.96162886,  2.6044915 ,  3.10598822,
        1.68469341, -1.17283117,  1.22145118,  1.72380054, -0.61452391,
        0.67288485, -1.50891888,  0.71395862, -1.15122455, -3.07094113])

In [124]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1) 
X = vectorizer.fit_transform(train_reviews)
clf1 = LogisticRegression(solver='lbfgs',C=1, random_state=1)
clf2 = LinearSVC(C=0.01)
clf3 = MultinomialNB(alpha=0.6)

eclf1 = VotingClassifier(estimators=[
        ('svm', clf2), ('nb', clf3)], voting='hard')# ('lr', clf1),
eclf1 = eclf1.fit(X, train_labels)

In [125]:
X_test= vectorizer.transform(test_reviews)

accuracy_score(eclf1.predict(X_test), test_labels)


/Users/davidvalensi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.88016

In [68]:
best_param_model=[]
lr_params = ParameterGrid({'C':[1e-4, 1e-3,0.01,0.1,0.5,1,3], 'solver':['newton-cg', 'lbfgs', 'liblinear']})
classifiers = [(LogisticRegression, lr_params)]
get_best_param(classifiers,X, train_labels) 

For classifier:  <class 'sklearn.linear_model.logistic.LogisticRegression'> :
The scores on cross validation are: [0.79733621 0.79649628 0.78948632]
Accuracy: 0.79 (+/- 0.01)
With the following params : {'C': 0.0001, 'solver': 'newton-cg'}
Mean Accuracy achieved :  0.7944396037371259
The scores on cross validation are: [0.79721622 0.79649628 0.78948632]
Accuracy: 0.79 (+/- 0.01)
With the following params : {'C': 0.0001, 'solver': 'lbfgs'}
Mean Accuracy achieved :  0.79439960693687
The scores on cross validation are: [0.79601632 0.79781617 0.78900624]
Accuracy: 0.79 (+/- 0.01)
With the following params : {'C': 0.0001, 'solver': 'liblinear'}
Mean Accuracy achieved :  0.7942795781330293
The scores on cross validation are: [0.85325174 0.85301176 0.84085454]
Accuracy: 0.85 (+/- 0.01)
With the following params : {'C': 0.001, 'solver': 'newton-cg'}
Mean Accuracy achieved :  0.849039345215321
The scores on cross validation are: [0.85325174 0.85301176 0.84109458]
Accuracy: 0.85 (+/- 0.01)
With 

KeyboardInterrupt: 